In [2]:
# handle bugs in some docker container: 'pip install -e .'  doesn't work 
from IPython.display import clear_output
try:
    import Flamingo
except ModuleNotFoundError:
    import sys 
    sys.path.append('..')
    import Flamingo
lang_encoder_path = "facebook/opt-125m"
tokenizer_path = lang_encoder_path
cache_dir = None 
model_config = dict(
    clip_vision_encoder_path="ViT-L-14",
    clip_vision_encoder_pretrained="openai",
    lang_encoder_path=lang_encoder_path,
    tokenizer_path=tokenizer_path,
    cross_attn_every_n_layers=1,
    cache_dir = cache_dir,
    lora_tuning=False,
    decoupled=True  
)

import torch
from Flamingo.lora_tuning import create_model_and_transforms 
from Flamingo.models.batchprocessor import DecoupledFlamingoBatchProcessor
from Flamingo.datasets.gtsrb import classes
from Flamingo.config.participants_property import dataset_config
from Flamingo.datasets import build_dataset
from Flamingo.inference.vis import show_pred_with_gt
from Flamingo.inference import post_process
import pandas as pd 
import re
from random import randint
clear_output()

In [3]:
# build model, image processor and tokenizer
model, image_processor, tokenizer = create_model_and_transforms(
    **model_config
)
print("Load state dict:")
state_dict = torch.load("/root/yunzhi/flamingo_retrieval/retrieval/work_dir/99/weight.pth")
keys1 = model.lang_encoder.gated_cross_attn_layers.load_state_dict(state_dict, strict=False)
keys2 = model.perceiver.load_state_dict(state_dict, strict=False)
dataset = build_dataset(
    dataset_config=dataset_config,
    vis_processor=image_processor,
    tokenizer=tokenizer)
model = model.to(device="cuda:2", dtype=torch.bfloat16)

batch_processor = DecoupledFlamingoBatchProcessor(cast_type='bf16', tokenizer=tokenizer)
model.eval()
clear_output()

In [5]:
[k for k in dataset[0]]

['vision_x',
 'category_name',
 'attributes_name',
 'ori_img_name',
 'file_name',
 'bbox',
 'area',
 'pth_file']

In [6]:
for data in dataset:
    bbox = data['bbox']
    x, y, w, h = bbox
    x, y, w, h = int(x), int(y), int(w), int(h)
    if x <0 or y < 0 or w < 0 or h < 0:
        continue 
    break 
data['bbox'], data['category_name'], data['attributes_name'], data['vision_x'].shape

([110.2237445779, 294.2411995329, 81.59200867539998, 116.00141808469999],
 'pedestrian',
 '0-50 obstruction',
 torch.Size([1, 1, 1, 256, 1024]))

In [120]:
def check_bbox(data):
    bbox = data['bbox']
    x, y, w, h = bbox
    x, y, w, h = int(x), int(y), int(w), int(h)
    if x <0 or y < 0 or w < 0 or h < 0:
        return False
    return True 
index = randint(0, len(dataset))
data = dataset[index] 
while not check_bbox(data):
    index = randint(0, len(dataset))
    data = dataset[index] 
index, data['bbox'], data['category_name'], data['attributes_name']

(5445,
 [645.2942545395, 482.2438995211, 52.380189045400016, 37.975637057899974],
 'car',
 '0-50 obstruction')

In [30]:
data = dataset[1506]

In [133]:
def post_process_participants(texts):
    if isinstance(texts, list):
        return [post_process_participants(text) for text in texts]
    elif isinstance(texts, str):
        texts = texts.split('\x7f')[0]
        texts = texts.split('�')[0]
        search_list = ['I\'m', 'guiAct',]
        result = texts
        for search_target in search_list:
            match = re.search(search_target, texts)
            if match:
                end_pos = match.start()
                # print(f"search_target={search_target}, end_pos={end_pos}")
                result = texts[:end_pos - 1]
        return result
    else:
        raise TypeError
with torch.inference_mode():
    output = batch_processor(model=model, batch=data['vision_x'], mode='test',
            text_prompt="<image>Output:", num_beams=3, max_new_tokens=20)
    print("before post process:", output)
    _output = post_process(output)
_output

before post process: ['Output:0-50 obstruction car�I\'m not a fan of the "I\'m not a fan']


['Output:0-50 obstruction car']

In [134]:
# re.search("I\'m not sure", output[0]).start()
'I\'' in output[0].split('�')[1]

True

In [15]:
post_process(output)

['Output:0-50 obstruction car']

In [1]:
def remove_special_characters(text):
    # 匹配 ASCII 特殊字符的正则表达式
    pattern = r'[^\x00-\x7f]+'
    
    # 使用空字符串替换匹配到的特殊字符
    result = re.sub(pattern, ' ', text)
    return result
pattern = r'[\x00-\x7f]+'
re.sub(pattern, ' ', output[0])

NameError: name 're' is not defined